In [31]:
pip install tensorboard

In [1]:
!pip install tensorflow keras opencv-python numpy


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
pip install tensorboard
!pip install tensorflow keras opencv-python numpy

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
import numpy as np
import cv2
import os

In [3]:
# Crear el ImageDataGenerator con aumento de datos
train_datagen = ImageDataGenerator(
    rescale=1./255,
     rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=5,
    #zoom_range=[0.7, 1.4],
    #horizontal_flip=True,
    #vertical_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
# Cargar las imágenes desde el directorio
train_generator = train_datagen.flow_from_directory(
    directory='C:/Users/Jair Trujillo/Downloads/Data_ABC/Train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 10400 images belonging to 26 classes.


In [6]:
validation_generator = val_datagen.flow_from_directory(
    directory='C:/Users/Jair Trujillo/Downloads/Data_ABC/Val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)
num_classes = len(train_generator.class_indices)

Found 2600 images belonging to 26 classes.


In [22]:
num_classes = len(train_generator.class_indices)

# Cargar el modelo base VGG16 preentrenado
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Añadir capas personalizadas al modelo
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=l2(0.01))(x)
x = Dropout(0.5)(x)
predictions = Dense(num_classes, activation='softmax', kernel_regularizer=l2(0.01))(x)

# Definir el modelo final
model = Model(inputs=base_model.input, outputs=predictions)

# Congelar las capas del modelo base
for layer in base_model.layers:
    layer.trainable = False

# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [13]:
# Definir callbacks para el primer entrenamiento
early_stopping_1 = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
model_checkpoint_1 = ModelCheckpoint('best_model_phase1.weights.h5', monitor='val_loss', save_best_only=True, verbose=1, save_weights_only=True)
tensorboardCNN_1 = TensorBoard(log_dir='logs/model_phase1')

callbacks_1 = [early_stopping_1, model_checkpoint_1, tensorboardCNN_1]

In [14]:
# Entrenar el modelo (primera fase: solo capas personalizadas)
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10,
    callbacks=callbacks_1
)

Epoch 1/10


C:\Users\Jair Trujillo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


325/325 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.3943 - loss: 4.5997
Epoch 1: val_loss improved from inf to 2.08377, saving model to best_model_phase1.weights.h5
325/325 ━━━━━━━━━━━━━━━━━━━━ 692s 2s/step - accuracy: 0.3949 - loss: 4.5952 - val_accuracy: 0.8634 - val_loss: 2.0838
Epoch 2/10


C:\Users\Jair Trujillo\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 2: val_loss did not improve from 2.08377
325/325 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 2.1139
Epoch 3/10
325/325 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7827 - loss: 2.1441
Epoch 3: val_loss improved from 2.08377 to 1.95779, saving model to best_model_phase1.weights.h5
325/325 ━━━━━━━━━━━━━━━━━━━━ 693s 2s/step - accuracy: 0.7827 - loss: 2.1440 - val_accuracy: 0.9005 - val_loss: 1.9578
Epoch 4/10

Epoch 4: val_loss did not improve from 1.95779
325/325 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 1.9638
Epoch 5/10
325/325 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.8377 - loss: 2.0229
Epoch 5: val_loss improved from 1.95779 to 1.90858, saving model to best_model_phase1.weights.h5
325/325 ━━━━━━━━━━━━━━━━━━━━ 689s 2s/step - accuracy: 0.8377 - loss: 2.0228 - val_accuracy: 0.9464 - val_loss: 1.9086
Epoch 6/10

Epoch 6: val_loss did not impro

In [23]:
# Descongelar algunas capas del modelo base y reentrenar (segunda fase)
for layer in base_model.layers[:15]:
    layer.trainable = False
for layer in base_model.layers[15:]:
    layer.trainable = True

#model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
# Definir callbacks para el segundo entrenamiento
early_stopping_2 = EarlyStopping(monitor='val_loss', patience=10, verbose=1)
model_checkpoint_2 = ModelCheckpoint('best_model_phase2.weights.h5', monitor='val_loss', save_best_only=True, verbose=1, save_weights_only=True)
tensorboardCNN_2 = TensorBoard(log_dir='logs/model_phase2')

callbacks_2 = [early_stopping_2, model_checkpoint_2, tensorboardCNN_2]


In [25]:
# Entrenar el modelo (segunda fase: descongelar algunas capas del modelo base)
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10,
    callbacks=callbacks_2
)

Epoch 1/10
325/325 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6834 - loss: 7.1426
Epoch 1: val_loss improved from inf to 3.47800, saving model to best_model_phase2.weights.h5
325/325 ━━━━━━━━━━━━━━━━━━━━ 785s 2s/step - accuracy: 0.6841 - loss: 7.1373 - val_accuracy: 1.0000 - val_loss: 3.4780
Epoch 2/10


C:\Users\Jair Trujillo\AppData\Local\Programs\Python\Python310\lib\contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 2: val_loss did not improve from 3.47800
325/325 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 3.4781
Epoch 3/10
325/325 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9993 - loss: 2.9790
Epoch 3: val_loss improved from 3.47800 to 1.84149, saving model to best_model_phase2.weights.h5
325/325 ━━━━━━━━━━━━━━━━━━━━ 804s 2s/step - accuracy: 0.9993 - loss: 2.9777 - val_accuracy: 0.9950 - val_loss: 1.8415
Epoch 4/10

Epoch 4: val_loss improved from 1.84149 to 1.82401, saving model to best_model_phase2.weights.h5
325/325 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 1.8240
Epoch 5/10
325/325 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9973 - loss: 1.6121
Epoch 5: val_loss improved from 1.82401 to 1.05669, saving model to best_model_phase2.weights.h5
325/325 ━━━━━━━━━━━━━━━━━━━━ 816s 3s/step - accuracy: 0.9973 - loss: 1.6115 - val_accuracy: 1.0000 - val_loss: 

In [42]:
model.save('Fine-Tuning VGG16.h5')

In [43]:
!mkdir carpeta_salida

In [44]:
!tensorflowjs_converter --input_format Fine-Tuning VGG16.h5 carpeta_salida

"tensorflowjs_converter" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.
